In [2]:
import pathlib
#import itertools

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import sklearn
from sklearn.model_selection import train_test_split

import cv2
#image = cv2.imread("path")
#rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Mean, SparseCategoricalAccuracy

from models.base_model import BaseModel
from models.res_model import ResModel



template = "numpy: v{np},  opencv: v{cv},  tensorflow: v{tf}"
print(template.format(np=np.__version__,
                      cv=cv2.__version__,
                      tf=tf.__version__))

numpy: v1.17.2,  opencv: v3.4.2,  tensorflow: v2.1.0-rc1


In [21]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

model = BaseModel()
print(model)
optimizer = Adam()
loss_object = SparseCategoricalCrossentropy()

train_loss = Mean(name="train_loss")
train_accuracy = SparseCategoricalAccuracy(name="train_accuracy")

test_loss = Mean(name="train_loss")
test_accuracy = SparseCategoricalAccuracy(name="train_accuracy")


@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


EPOCHS = 5

for e in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(images, labels)

    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print(template.format(e,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100))





If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0, Loss: 0.13338521122932434, Accuracy: 95.98833465576172, Test Loss: 0.005342455115169287, Test Accuracy: 100.0
Epoch 1, Loss: 0.08781207352876663, Accuracy: 97.34916687011719, Test Loss: 0.007974609732627869, Test Accuracy: 100.0
Epoch 2, Loss: 0.06621803343296051, Accuracy: 97.9911117553711, Test Loss: 0.008847370743751526, Test Accuracy: 100.0
Epoch 3, Loss: 0.0533098541200161, Accuracy: 98.37166595458984, Test Loss: 0.01492581982165575, Test Accuracy: 99.21875
Epoch 4, Loss: 0.044648073613643

In [22]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

model = ResModel()
print(model)
optimizer = Adam()
loss_object = SparseCategoricalCrossentropy()

train_loss = Mean(name="train_loss")
train_accuracy = SparseCategoricalAccuracy(name="train_accuracy")

test_loss = Mean(name="train_loss")
test_accuracy = SparseCategoricalAccuracy(name="train_accuracy")


@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)


@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


EPOCHS = 5

for e in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(images, labels)

    template = "Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}"
    print(template.format(e,
        train_loss.result(),
        train_accuracy.result() * 100,
        test_loss.result(),
        test_accuracy.result() * 100))






If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0, Loss: 0.1406075656414032, Accuracy: 95.80166625976562, Test Loss: 0.07978011667728424, Test Accuracy: 96.875
Epoch 1, Loss: 0.09293968230485916, Accuracy: 97.19833374023438, Test Loss: 0.04713798314332962, Test Accuracy: 98.4375
Epoch 2, Loss: 0.06957311928272247, Accuracy: 97.89277648925781, Test Loss: 0.03150924667716026, Test Accuracy: 98.95832824707031
Epoch 3, Loss: 0.055671464651823044, Accuracy: 98.30458068847656, Test Loss: 0.02458799257874489, Test Accuracy: 99.21875
Epoch 4, Loss: 0.0